In [16]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import html5lib

SCORE_DIRECTORY = "nba_data/scores"
STANDINGS_DIRECTORY = "nba_data/standings"

In [17]:
current_data = 'nba_data_up_to.csv'
current_data = pd.read_csv(current_data)

In [18]:
all_box_scores = os.listdir(SCORE_DIRECTORY)
all_box_scores = [os.path.join(SCORE_DIRECTORY, f) for f in all_box_scores if f.endswith(".html")]

In [19]:
# Parse HTML file for each box score

def parse_box_score(box_score):
    
    print(f"Parsing the {box_score} box score")
    with open(box_score) as f:
        html = f.read()
    
    # Decompose time series into seasonal, trend and irregular components
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    
    return soup

In [20]:
# Open the information for each season

def open_season_information(soup):
    print("Opening and returning the season information")
    extracted_nav_list = soup.select("#bottom_nav_container")[0]
    all_hrefs = [a["href"] for a in extracted_nav_list.find_all('a')]
    
    season = os.path.basename(all_hrefs[1]).split("_")[0]
    return season

In [21]:
# Display the line score: a summary of the scoring in a game displayed in a horizontal table

def display_line_score(soup):
    print("Displaying line score")
    line_score = pd.read_html(str(soup), attrs={'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]
    
    return line_score

In [22]:
# Read statistics

def read_stats(soup, team, stat):
    print(f"Reading stats of team {team}")
    
    df = pd.read_html(str(soup), attrs={'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [23]:
# Extract files for specific seasons (optional)

standings_files = os.listdir(STANDINGS_DIRECTORY)

# Only process files for the 2022-23 and 2023-24 seasons
standings_files_2023_2024 = [f for f in standings_files if 
    '2023' in f or 
    '2024' in f or 
    ('2022' in f and ('october' in f or 'november' in f or 'december' in f))
]

standings_files_2023_2024

['NBA_2022_games-october.html',
 'NBA_2024_games-may.html',
 'NBA_2023_games-may.html',
 'NBA_2023_games-april.html',
 'NBA_2023_games-january.html',
 'NBA_2024_games-march.html',
 'NBA_2024_games-november.html',
 'NBA_2024_games-december.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2024_games-october.html',
 'NBA_2023_games-december.html',
 'NBA_2022_games-november.html',
 'NBA_2023_games-february.html',
 'NBA_2024_games-february.html',
 'NBA_2024_games-january.html',
 'NBA_2024_games-april.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-march.html',
 'NBA_2023_games-october.html',
 'NBA_2024_games-june.html']

In [26]:
# Setup dataset

games_list = []
base_columns = None

In [27]:
# Fill info into dataset

for box_score_file in all_box_scores:
    print(f"Processing box score file: {box_score_file}")
    soup = parse_box_score(box_score_file)

    line_score_df = display_line_score(soup)
    teams = list(line_score_df["team"])

    team_summaries = []
    for team in teams:
        print(f"Processing team: {team}")
        basic_stats = read_stats(soup, team, "basic")
        advanced_stats = read_stats(soup, team, "advanced")

        total_stats = pd.concat([basic_stats.iloc[-1, :], advanced_stats.iloc[-1, :]])
        total_stats.index = total_stats.index.str.lower()

        max_stats = pd.concat([basic_stats.iloc[:-1].max(), advanced_stats.iloc[:-1].max()])
        max_stats.index = max_stats.index.str.lower() + "_max"

        team_summary = pd.concat([total_stats, max_stats])
        
        if base_columns is None:
            base_columns = list(team_summary.index.drop_duplicates(keep="first"))
            base_columns = [col for col in base_columns if "bpm" not in col]
        
        team_summary = team_summary[base_columns]
        
        team_summaries.append(team_summary)
    
    combined_summary = pd.concat(team_summaries, axis=1).T

    game_summary = pd.concat([combined_summary, line_score_df], axis=1)

    game_summary["home"] = [0, 1]

    game_summary_opp = game_summary.iloc[::-1].reset_index()
    game_summary_opp.columns += "_opp"

    full_game_summary = pd.concat([game_summary, game_summary_opp], axis=1)
    full_game_summary["season"] = open_season_information(soup)
    
    full_game_summary["date"] = os.path.basename(box_score_file)[:8]
    full_game_summary["date"] = pd.to_datetime(full_game_summary["date"], format="%Y%m%d")
    
    full_game_summary["won"] = full_game_summary["total"] > full_game_summary["total_opp"]
    games_list.append(full_game_summary)
    
    if len(games_list) % 100 == 0:
        print(f"{len(games_list)} / {len(all_box_scores)}")

print("Completed data parsing")

Processing box score file: nba_data/scores/202211070MIA.html
Parsing the nba_data/scores/202211070MIA.html box score
Displaying line score
Processing team: POR
Reading stats of team POR
Reading stats of team POR
Processing team: MIA
Reading stats of team MIA
Reading stats of team MIA
Opening and returning the season information
Processing box score file: nba_data/scores/202212140DAL.html
Parsing the nba_data/scores/202212140DAL.html box score
Displaying line score
Processing team: CLE
Reading stats of team CLE
Reading stats of team CLE
Processing team: DAL
Reading stats of team DAL
Reading stats of team DAL
Opening and returning the season information
Processing box score file: nba_data/scores/202212070TOR.html
Parsing the nba_data/scores/202212070TOR.html box score
Displaying line score
Processing team: LAL
Reading stats of team LAL
Reading stats of team LAL
Processing team: TOR
Reading stats of team TOR
Reading stats of team TOR
Opening and returning the season information
Processing

Opening and returning the season information
Processing box score file: nba_data/scores/202212140ORL.html
Parsing the nba_data/scores/202212140ORL.html box score
Displaying line score
Processing team: ATL
Reading stats of team ATL
Reading stats of team ATL
Processing team: ORL
Reading stats of team ORL
Reading stats of team ORL
Opening and returning the season information
Processing box score file: nba_data/scores/202311220PHO.html
Parsing the nba_data/scores/202311220PHO.html box score
Displaying line score
Processing team: GSW
Reading stats of team GSW
Reading stats of team GSW
Processing team: PHO
Reading stats of team PHO
Reading stats of team PHO
Opening and returning the season information
Processing box score file: nba_data/scores/202312110HOU.html
Parsing the nba_data/scores/202312110HOU.html box score
Displaying line score
Processing team: SAS
Reading stats of team SAS
Reading stats of team SAS
Processing team: HOU
Reading stats of team HOU
Reading stats of team HOU
Opening an

Reading stats of team NYK
Opening and returning the season information
Processing box score file: nba_data/scores/202402120MEM.html
Parsing the nba_data/scores/202402120MEM.html box score
Displaying line score
Processing team: NOP
Reading stats of team NOP
Reading stats of team NOP
Processing team: MEM
Reading stats of team MEM
Reading stats of team MEM
Opening and returning the season information
Processing box score file: nba_data/scores/202303080NOP.html
Parsing the nba_data/scores/202303080NOP.html box score
Displaying line score
Processing team: DAL
Reading stats of team DAL
Reading stats of team DAL
Processing team: NOP
Reading stats of team NOP
Reading stats of team NOP
Opening and returning the season information
Processing box score file: nba_data/scores/202312050LAL.html
Parsing the nba_data/scores/202312050LAL.html box score
Displaying line score
Processing team: PHO
Reading stats of team PHO
Reading stats of team PHO
Processing team: LAL
Reading stats of team LAL
Reading st

Reading stats of team BRK
Opening and returning the season information
Processing box score file: nba_data/scores/202402230PHI.html
Parsing the nba_data/scores/202402230PHI.html box score
Displaying line score
Processing team: CLE
Reading stats of team CLE
Reading stats of team CLE
Processing team: PHI
Reading stats of team PHI
Reading stats of team PHI
Opening and returning the season information
Processing box score file: nba_data/scores/202402080IND.html
Parsing the nba_data/scores/202402080IND.html box score
Displaying line score
Processing team: GSW
Reading stats of team GSW
Reading stats of team GSW
Processing team: IND
Reading stats of team IND
Reading stats of team IND
Opening and returning the season information
Processing box score file: nba_data/scores/202403150UTA.html
Parsing the nba_data/scores/202403150UTA.html box score
Displaying line score
Processing team: ATL
Reading stats of team ATL
Reading stats of team ATL
Processing team: UTA
Reading stats of team UTA
Reading st

Processing team: WAS
Reading stats of team WAS
Reading stats of team WAS
Opening and returning the season information
100 / 2639
Processing box score file: nba_data/scores/202401130ATL.html
Parsing the nba_data/scores/202401130ATL.html box score
Displaying line score
Processing team: WAS
Reading stats of team WAS
Reading stats of team WAS
Processing team: ATL
Reading stats of team ATL
Reading stats of team ATL
Opening and returning the season information
Processing box score file: nba_data/scores/202211120WAS.html
Parsing the nba_data/scores/202211120WAS.html box score
Displaying line score
Processing team: UTA
Reading stats of team UTA
Reading stats of team UTA
Processing team: WAS
Reading stats of team WAS
Reading stats of team WAS
Opening and returning the season information
Processing box score file: nba_data/scores/202212100CLE.html
Parsing the nba_data/scores/202212100CLE.html box score
Displaying line score
Processing team: OKC
Reading stats of team OKC
Reading stats of team OKC

...

Displaying line score
Processing team: MIL
Reading stats of team MIL
Reading stats of team MIL
Processing team: MEM
Reading stats of team MEM
Reading stats of team MEM
Opening and returning the season information
Processing box score file: nba_data/scores/202211130NYK.html
Parsing the nba_data/scores/202211130NYK.html box score
Displaying line score
Processing team: OKC
Reading stats of team OKC
Reading stats of team OKC
Processing team: NYK
Reading stats of team NYK
Reading stats of team NYK
Opening and returning the season information
Processing box score file: nba_data/scores/202211070GSW.html
Parsing the nba_data/scores/202211070GSW.html box score
Displaying line score
Processing team: SAC
Reading stats of team SAC
Reading stats of team SAC
Processing team: GSW
Reading stats of team GSW
Reading stats of team GSW
Opening and returning the season information
Processing box score file: nba_data/scores/202301130SAS.html
Parsing the nba_data/scores/202301130SAS.html box score
Displaying

Displaying line score
Processing team: DEN
Reading stats of team DEN
Reading stats of team DEN
Processing team: HOU
Reading stats of team HOU
Reading stats of team HOU
Opening and returning the season information
Processing box score file: nba_data/scores/202401120SAS.html
Parsing the nba_data/scores/202401120SAS.html box score
Displaying line score
Processing team: CHO
Reading stats of team CHO
Reading stats of team CHO
Processing team: SAS
Reading stats of team SAS
Reading stats of team SAS
Opening and returning the season information
Processing box score file: nba_data/scores/202401260TOR.html
Parsing the nba_data/scores/202401260TOR.html box score
Displaying line score
Processing team: LAC
Reading stats of team LAC
Reading stats of team LAC
Processing team: TOR
Reading stats of team TOR
Reading stats of team TOR
Opening and returning the season information
Processing box score file: nba_data/scores/202301260BOS.html
Parsing the nba_data/scores/202301260BOS.html box score
Displaying

Processing team: LAC
Reading stats of team LAC
Reading stats of team LAC
Processing team: IND
Reading stats of team IND
Reading stats of team IND
Opening and returning the season information
Processing box score file: nba_data/scores/202211260PHO.html
Parsing the nba_data/scores/202211260PHO.html box score
Displaying line score
Processing team: UTA
Reading stats of team UTA
Reading stats of team UTA
Processing team: PHO
Reading stats of team PHO
Reading stats of team PHO
Opening and returning the season information
Processing box score file: nba_data/scores/202212150HOU.html
Parsing the nba_data/scores/202212150HOU.html box score
Displaying line score
Processing team: MIA
Reading stats of team MIA
Reading stats of team MIA
Processing team: HOU
Reading stats of team HOU
Reading stats of team HOU
Opening and returning the season information
Processing box score file: nba_data/scores/202301170MIL.html
Parsing the nba_data/scores/202301170MIL.html box score
Displaying line score
Processing

Displaying line score
Processing team: MIL
Reading stats of team MIL
Reading stats of team MIL
Processing team: IND
Reading stats of team IND
Reading stats of team IND
Opening and returning the season information
2600 / 2639
Processing box score file: nba_data/scores/202304040CHI.html
Parsing the nba_data/scores/202304040CHI.html box score
Displaying line score
Processing team: ATL
Reading stats of team ATL
Reading stats of team ATL
Processing team: CHI
Reading stats of team CHI
Reading stats of team CHI
Opening and returning the season information
Processing box score file: nba_data/scores/202311170LAC.html
Parsing the nba_data/scores/202311170LAC.html box score
Displaying line score
Processing team: HOU
Reading stats of team HOU
Reading stats of team HOU
Processing team: LAC
Reading stats of team LAC
Reading stats of team LAC
Opening and returning the season information
Processing box score file: nba_data/scores/202302250CHO.html
Parsing the nba_data/scores/202302250CHO.html box scor

Displaying line score
Processing team: PHO
Reading stats of team PHO
Reading stats of team PHO
Processing team: MEM
Reading stats of team MEM
Reading stats of team MEM
Opening and returning the season information
Processing box score file: nba_data/scores/202404050CHI.html
Parsing the nba_data/scores/202404050CHI.html box score
Displaying line score
Processing team: NYK
Reading stats of team NYK
Reading stats of team NYK
Processing team: CHI
Reading stats of team CHI
Reading stats of team CHI
Opening and returning the season information
Processing box score file: nba_data/scores/202401180TOR.html
Parsing the nba_data/scores/202401180TOR.html box score
Displaying line score
Processing team: CHI
Reading stats of team CHI
Reading stats of team CHI
Processing team: TOR
Reading stats of team TOR
Reading stats of team TOR
Opening and returning the season information
Processing box score file: nba_data/scores/202301020IND.html
Parsing the nba_data/scores/202301020IND.html box score
Displaying

In [28]:
# Combine new games into a DataFrame
new_games_df = pd.concat(games_list, ignore_index=True)
new_games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5273,240.0,240.0,47.0,89.0,0.528,14.0,31.0,0.452,19.0,25.0,...,41.0,45.3,162.0,135.0,DEN,126,0,2023,2022-12-28,True
5274,240.0,240.0,43.0,87.0,0.494,10.0,27.0,0.370,21.0,27.0,...,33.6,31.0,163.0,123.0,SAS,110,1,2024,2023-11-10,True
5275,240.0,240.0,43.0,92.0,0.467,12.0,34.0,0.353,12.0,19.0,...,51.5,33.0,200.0,112.0,MIN,117,0,2024,2023-11-10,False
5276,240.0,240.0,42.0,82.0,0.512,12.0,27.0,0.444,12.0,19.0,...,53.2,28.0,146.0,126.0,TOR,121,1,2024,2023-11-24,False


In [41]:
new_games_df.to_csv("nba_data_extension.csv")
current_data

,Unnamed: 0,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,17767,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17768,17768,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17769,17769,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17770,17770,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [42]:
# append data frame to CSV file
new_games_df.to_csv('nba_data_up_to copy.csv', mode='a', index=False, header=True)
 
# print message
print("Data appended successfully.")

Data appended successfully.
